## Escuela de Ingeniería en Computación, ITCR 
## Aprendizaje automático

## Preprocesamiento de datos categóricos

**Profesora: María Auxiliadora Mora**


### Introducción

Una variable es una función que asocia a cada elemento de la población la medición de una característica, particularmente de la característica que se desea observar. La variables **pueden ser númericas o categóricas**.

![](../imagenes/variables_tipos.png)


Las variables categóricas representan tipos de datos agrupados en clases,  generalmente se representan en formato de texto que corresponde a **nombres de categorías y son finitas en número**. Por ejemplo: 

- Nivel de statisfacción: Muy satisfecho, satisfecho, indiferente, insatisfecho, my insatisfecho
- Rangos de salario: 1-500K, 501K-1000k, 1001-1500k, etc
- Nivel de educación de las personas: Inicial, primaria, secundaria, superior.
- Provincias: Alajuela, Heredia, San José, Cartago, Puntarenas, Limón, Guanacaste. 
- Géneros de vídeo juegos
- País
- Sexo

Existen al menos dos grandes clases de atributos categóricos:

- **Atributos nominales**: Las categorías no tienen un orden inherente, por ejemplo país, sexo, provincia, género de las películas, entre otros.

- **Atributos ordinales**: Las categorías tienen un orden, por ejemplo nivel de satisfacción, rangos de salarios, tallas de ropa, entre otros.

Si bien se han realizado muchos avances en varias bibliotecas de aprendizaje automático para aceptar tipos de datos categóricos complejos. **Normalmente, los algoritmos esperan que los datos sean numéricos** por lo que cualquier flujo de trabajo de aprendizaje automático implica alguna forma de **transformación de estos valores categóricos a datos numéricos**. **Por lo general se transforman primero en etiquetas numéricas y luego se debe aplicar algún esquema de codificación en estos valores**. 

En el caso de **categoría nominales, luego de convertirlas a representaciones numéricas hay que codificarlas para evitar que los modelos interpreten esas representaciones como valor continuos**. Por ejemplo, el caso de los géneros de los vídeo juegos, si se incluye directamente un código como una característica en un modelo de aprendizaje automático, este lo podría considerar como una característica numérica continua lo que podría no generar los resultados esperados. Por ejemplo, un modelo podía interpretar que un tipo de juego con código mayor es mejor que otros tipos. Por lo tanto, se requiere una capa adicional de codificación donde se crean características ficticias para cada categoría del atributo. Bibliotecas como [category_encoders](https://contrib.scikit-learn.org/category_encoders/index.html) no requieren el paso intermedio.

El objetivo de este documento es introducir algunos de los métodos existentes para codificar datos categóricos y presentar algunos ejemplos. 


### Los datos usado en los ejemplos fueron generados artificialmente o tomados de:

- https://www.kaggle.com/gregorut/videogamesales

- https://www.kaggle.com/arashnic/hr-analytics-job-change-of-data-scientists


In [1]:
#!pip install category_encoders

In [29]:
#Libraries
import pandas as pd
import numpy as np

# Simple encoder by sklearn 
from sklearn.preprocessing import OneHotEncoder, LabelEncoder

import category_encoders as ce

## Ejemplos 

Datos: Videogame Sales

El conjunto de datos contiene una lista de videojuegos con ventas superiores a 100.000 copias que fue generado por vgchartz.com. Consta de más de 16,000 registros. 

Las columnas del conjunto de datos son las siguientes:

Rank - Ranking of overall sales

Name - The games name

Platform - Platform of the games release (i.e. PC,PS4, etc.)

Year - Year of the game's release

Genre - Genre of the game

Publisher - Publisher of the game

NA_Sales - Sales in North America (in millions)

EU_Sales - Sales in Europe (in millions)

JP_Sales - Sales in Japan (in millions)

Other_Sales - Sales in the rest of the world (in millions)

Global_Sales - Total worldwide sales.


In [30]:
# Load and display data
vg_df = pd.read_csv('../../Data/vgsales.csv', encoding='utf-8')

print(vg_df.columns)

Index(['Rank', 'Name', 'Platform', 'Year', 'Genre', 'Publisher', 'NA_Sales',
       'EU_Sales', 'JP_Sales', 'Other_Sales', 'Global_Sales'],
      dtype='object')


In [31]:
print(vg_df[['Rank','Name', 'Platform', 'Genre', 'Publisher', 'Global_Sales']].head(20))


    Rank                                          Name Platform         Genre  \
0      1                                    Wii Sports      Wii        Sports   
1      2                             Super Mario Bros.      NES      Platform   
2      3                                Mario Kart Wii      Wii        Racing   
3      4                             Wii Sports Resort      Wii        Sports   
4      5                      Pokemon Red/Pokemon Blue       GB  Role-Playing   
5      6                                        Tetris       GB        Puzzle   
6      7                         New Super Mario Bros.       DS      Platform   
7      8                                      Wii Play      Wii          Misc   
8      9                     New Super Mario Bros. Wii      Wii      Platform   
9     10                                     Duck Hunt      NES       Shooter   
10    11                                    Nintendogs       DS    Simulation   
11    12                    

In [32]:
# Datos únicos
print('=======================================================')
print('Plataforma', pd.unique(vg_df['Platform']))
print('=======================================================')
print('Género', pd.unique(vg_df['Genre']))
print('=======================================================')
# Conteo de registros
print('Editorial', pd.value_counts(vg_df['Publisher']))

Plataforma ['Wii' 'NES' 'GB' 'DS' 'X360' 'PS3' 'PS2' 'SNES' 'GBA' '3DS' 'PS4' 'N64'
 'PS' 'XB' 'PC' '2600' 'PSP' 'XOne' 'GC' 'WiiU' 'GEN' 'DC' 'PSV' 'SAT'
 'SCD' 'WS' 'NG' 'TG16' '3DO' 'GG' 'PCFX']
Género ['Sports' 'Platform' 'Racing' 'Role-Playing' 'Puzzle' 'Misc' 'Shooter'
 'Simulation' 'Action' 'Fighting' 'Adventure' 'Strategy']
Editorial Electronic Arts                 1351
Activision                       975
Namco Bandai Games               932
Ubisoft                          921
Konami Digital Entertainment     832
                                ... 
Warp                               1
New                                1
Elite                              1
Evolution Games                    1
UIG Entertainment                  1
Name: Publisher, Length: 578, dtype: int64


## 1) Ejemplos usando Category Encoders

### One-Hot Encoding

Esta técnica de codificación de datos categóricos es utilizada cuando las **características son nominales**. Para cada categoría de una característica se crea una nueva variable en el dataset (es decir una columna). Cada categoría se asigna con una variable binaria que contiene 0 o 1. Aquí, 0 representa la ausencia y 1 representa la presencia de esa categoría. 

In [33]:
# Example (5 provinces) en 7 registros.

data=pd.DataFrame({'Province':['San José','Alajuela','Cartago','Puntarenas','Guanacaste',
                           'San José','Cartago', 'Heredia', 'Alajuela', 'Puntarenas', 'Limón']})

#Original Data
data

,Province
0,San José
1,Alajuela
2,Cartago
3,Puntarenas
4,Guanacaste
5,San José
6,Cartago
7,Heredia
8,Alajuela
9,Puntarenas


In [34]:
# An instance of the encoder is created 
encoder=ce.OneHotEncoder(cols='Province',handle_unknown='return_nan',return_df=True,use_cat_names=True)

#Fit and transform Data
data_encoded = encoder.fit_transform(data)

data_encoded

,Province_San José,Province_Alajuela,Province_Cartago,Province_Puntarenas,Province_Guanacaste,Province_Heredia,Province_Limón
0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,1.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,1.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,1.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,1.0,0.0,0.0
5,1.0,0.0,0.0,0.0,0.0,0.0,0.0
6,0.0,0.0,1.0,0.0,0.0,0.0,0.0
7,0.0,0.0,0.0,0.0,0.0,1.0,0.0
8,0.0,1.0,0.0,0.0,0.0,0.0,0.0
9,0.0,0.0,0.0,1.0,0.0,0.0,0.0


### Ventajas
- Muy fácil de implementar y manejar valores de columnas categóricas.

### Desventajas

- No es útil cuando los datos tienen muchas columnas categóricas.
- No es útil si la columna categórica tiene muchas categorías, debido a que se pueden agregar muchas características al conjunto de datos.

### Dummy Encoder

Genera un resultado parecido al One-Hot Encoding. El Dummy Encoder realizar una pequeña mejora y utiliza N-1 características para representar N categorías. 

In [35]:
#encode the data
data_encoded=pd.get_dummies(data=data,drop_first=True)
data_encoded

,Province_Cartago,Province_Guanacaste,Province_Heredia,Province_Limón,Province_Puntarenas,Province_San José
0,0,0,0,0,0,1
1,0,0,0,0,0,0
2,1,0,0,0,0,0
3,0,0,0,0,1,0
4,0,1,0,0,0,0
5,0,0,0,0,0,1
6,1,0,0,0,0,0
7,0,0,1,0,0,0
8,0,0,0,0,0,0
9,0,0,0,0,1,0


### Ventajas y desventajas 

Igual que la técnica One-Hot Encoding

### Effect Enconding

Esta técnica de codificación también se conoce como Deviation Encoding or Sum Encoding. Effect Encoding es similar a la codificación Dummy, con una pequeña diferencia, en la codificación Dummy, se usa 0 y 1 para representar los datos, pero en la codificación Effect, se usa tres valores, es decir, 1,0 y -1. 

La fila que contiene solo 0 en la codificación Dummy se codifica como -1 en la codificación Effect. 

In [36]:
# An instance of the encoder is created 
encoder=ce.sum_coding.SumEncoder(cols='Province',verbose=False,)

data_encoded=encoder.fit_transform(data)

data_encoded

/home/mmora/anaconda3/envs/ml_env/lib/python3.9/site-packages/category_encoders/base_contrast_encoder.py:126: FutureWarning: Intercept column might not be added anymore in future releases (c.f. issue #370)
  warnings.warn("Intercept column might not be added anymore in future releases (c.f. issue #370)",
/home/mmora/anaconda3/envs/ml_env/lib/python3.9/site-packages/category_encoders/base_contrast_encoder.py:126: FutureWarning: Intercept column might not be added anymore in future releases (c.f. issue #370)
  warnings.warn("Intercept column might not be added anymore in future releases (c.f. issue #370)",


,intercept,Province_0,Province_1,Province_2,Province_3,Province_4,Province_5
0,1,1.0,0.0,0.0,0.0,0.0,0.0
1,1,0.0,1.0,0.0,0.0,0.0,0.0
2,1,0.0,0.0,1.0,0.0,0.0,0.0
3,1,0.0,0.0,0.0,1.0,0.0,0.0
4,1,0.0,0.0,0.0,0.0,1.0,0.0
5,1,1.0,0.0,0.0,0.0,0.0,0.0
6,1,0.0,0.0,1.0,0.0,0.0,0.0
7,1,0.0,0.0,0.0,0.0,0.0,1.0
8,1,0.0,1.0,0.0,0.0,0.0,0.0
9,1,0.0,0.0,0.0,1.0,0.0,0.0


### Ventajas y desventajas 

Igual que la técnica One-Hot Encoding.

### Hash Encoder

El hash implica calcular un fingerprint matemático de longitud fija a partir de los datos de entrada, estos pueden ser de cualquier tamaño. A diferencia de la codificación, el hash no se puede revertir, es decir, no es posible tomar un valor hash y convertirlo de nuevo a los datos originales. El hash se usa comúnmente para verificar la integridad de los datos, es conocido como suma de verificación. Si dos datos idénticos se procesan con la misma función hash, el hash resultante será idéntico. Si los dos datos son diferentes, los hashes resultantes serán diferentes y únicos.

De forma predeterminada, el codificador Hash utiliza el algoritmo hash md5, pero el usuario puede seleccionar otros algoritmos. 

In [37]:
# Create dataframe
data=pd.DataFrame({'Month':['January','April','March','April','Februay','June','July','June','September']})

# An instance of the encoder is created 
encoder=ce.HashingEncoder(cols='Month',n_components=4)

data

,Month
0,January
1,April
2,March
3,April
4,Februay
5,June
6,July
7,June
8,September


In [38]:
# Encoder is applied to data 
data_encoded = encoder.fit_transform(data)
data_encoded

,col_0,col_1,col_2,col_3
0,0,0,1,0
1,0,1,0,0
2,1,0,0,0
3,0,1,0,0
4,0,0,0,1
5,0,1,0,0
6,1,0,0,0
7,0,1,0,0
8,1,0,0,0


### Ordinal encoder 

Cuando las variables categóricas son ordinales, el enfoque más sencillo es reemplazar cada etiqueta/categoría por algún número ordinal basado en los rangos. Por ejemplo, para tallas de prendas de vestir small, medium, large se asignan los códigos 1,2 y 3 repectivamente.


In [39]:
# Create dataframe
data=pd.DataFrame({'Size':['small','large','medium','small','medium','large','medium','medium','small']})

print(data)

# Create a dictionary with key as category and values with its rank. 
ClassDict =   {   'small':1,
                    'medium':2,
                     'large':3,              
                }
# Create a new column and map the ordinal column with the created dictionary.
data['Size_code'] = data.Size.map(ClassDict)
# Display result
      
print(data)

     Size
0   small
1   large
2  medium
3   small
4  medium
5   large
6  medium
7  medium
8   small
     Size  Size_code
0   small          1
1   large          3
2  medium          2
3   small          1
4  medium          2
5   large          3
6  medium          2
7  medium          2
8   small          1


### Ventaja
- La forma más fácil de manejar tipos de datos categóricos ordinales.

### Desventaja
- No es buena opción para las características de tipo nominal.

## 2) Ejemplos usando SkLearn

### Transformando atributos nominales

#### Paso 1: se asignan categorías numéricas

In [40]:
# Data from Kaggle: video game data set. 

# Load and display data
vg_df = pd.read_csv('../../Data/vgsales.csv', encoding='utf-8')

print(vg_df[['Name', 'Platform', 'Year', 'Genre', 'Publisher']].head(10))

                        Name Platform    Year         Genre Publisher
0                 Wii Sports      Wii  2006.0        Sports  Nintendo
1          Super Mario Bros.      NES  1985.0      Platform  Nintendo
2             Mario Kart Wii      Wii  2008.0        Racing  Nintendo
3          Wii Sports Resort      Wii  2009.0        Sports  Nintendo
4   Pokemon Red/Pokemon Blue       GB  1996.0  Role-Playing  Nintendo
5                     Tetris       GB  1989.0        Puzzle  Nintendo
6      New Super Mario Bros.       DS  2006.0      Platform  Nintendo
7                   Wii Play      Wii  2006.0          Misc  Nintendo
8  New Super Mario Bros. Wii      Wii  2009.0      Platform  Nintendo
9                  Duck Hunt      NES  1984.0       Shooter  Nintendo


In [41]:
# Select different values of a category 

genres = np.unique(vg_df['Genre'])

print("Cantidad de valores distintos de géneros", genres.shape)
print(genres)

Cantidad de valores distintos de géneros (12,)
['Action' 'Adventure' 'Fighting' 'Misc' 'Platform' 'Puzzle' 'Racing'
 'Role-Playing' 'Shooter' 'Simulation' 'Sports' 'Strategy']


In [42]:
print(genres)

['Action' 'Adventure' 'Fighting' 'Misc' 'Platform' 'Puzzle' 'Racing'
 'Role-Playing' 'Shooter' 'Simulation' 'Sports' 'Strategy']


In [43]:
# A different number label is assigned to each category using scikit-learn.   
labelEnc = LabelEncoder()

genre_labels = labelEnc.fit_transform(vg_df['Genre'])
labels = labelEnc.fit_transform(vg_df['Genre'])
genre_mappings = {index: label for index, label in 
                  enumerate(labelEnc.classes_)}
print(genre_mappings)

{0: 'Action', 1: 'Adventure', 2: 'Fighting', 3: 'Misc', 4: 'Platform', 5: 'Puzzle', 6: 'Racing', 7: 'Role-Playing', 8: 'Shooter', 9: 'Simulation', 10: 'Sports', 11: 'Strategy'}


In [44]:
print(genre_labels)

[10  4  6 ...  6  5  4]


In [45]:
# Classes of LabelEncoder
print(labelEnc.classes_)

['Action' 'Adventure' 'Fighting' 'Misc' 'Platform' 'Puzzle' 'Racing'
 'Role-Playing' 'Shooter' 'Simulation' 'Sports' 'Strategy']


In [46]:
# The column of labels in the original dataframe is added 

vg_df['GenreLabel'] = genre_labels
print(vg_df[['Name', 'Platform', 'Year', 'Genre', 'GenreLabel']].head(10))

                        Name Platform    Year         Genre  GenreLabel
0                 Wii Sports      Wii  2006.0        Sports          10
1          Super Mario Bros.      NES  1985.0      Platform           4
2             Mario Kart Wii      Wii  2008.0        Racing           6
3          Wii Sports Resort      Wii  2009.0        Sports          10
4   Pokemon Red/Pokemon Blue       GB  1996.0  Role-Playing           7
5                     Tetris       GB  1989.0        Puzzle           5
6      New Super Mario Bros.       DS  2006.0      Platform           4
7                   Wii Play      Wii  2006.0          Misc           3
8  New Super Mario Bros. Wii      Wii  2009.0      Platform           4
9                  Duck Hunt      NES  1984.0       Shooter           8


#### Transformando atributos ordinales

Los atributos ordinales son atributos categóricos con un sentido de orden entre los valores. 

In [47]:
# Load data from Kaggel: Job Change dataset
job_df = pd.read_csv('categorical_data/JobChange_DataScientists/aug_train.csv', encoding='utf-8')

print(job_df.head())

# Delete the records with null values in the columns 
# to be use in the example. 
job_df = job_df[pd.notnull(job_df['education_level'])]
job_df = job_df[pd.notnull(job_df['gender'])]
job_df.dropna(subset = job_df.columns, inplace=True)

np.unique(job_df['education_level'])


   enrollee_id      city  city_development_index gender  \
0         8949  city_103                   0.920   Male   
1        29725   city_40                   0.776   Male   
2        11561   city_21                   0.624    NaN   
3        33241  city_115                   0.789    NaN   
4          666  city_162                   0.767   Male   

       relevent_experience enrolled_university education_level  \
0  Has relevent experience       no_enrollment        Graduate   
1   No relevent experience       no_enrollment        Graduate   
2   No relevent experience    Full time course        Graduate   
3   No relevent experience                 NaN        Graduate   
4  Has relevent experience       no_enrollment         Masters   

  major_discipline experience company_size    company_type last_new_job  \
0             STEM        >20          NaN             NaN            1   
1             STEM         15        50-99         Pvt Ltd           >4   
2             STEM     

array(['Graduate', 'Masters', 'Phd'], dtype=object)

In [48]:
# Educational level processing in three classes 
educationalLevel_ord_map = {'Graduate': 1, 'Masters': 2, 'Phd': 3}

job_df['education_level_label'] = job_df['education_level'].map(educationalLevel_ord_map)

print(job_df[['enrollee_id', 'education_level', 'education_level_label']].head(20))

    enrollee_id education_level  education_level_label
1         29725        Graduate                      1
4           666         Masters                      2
7           402        Graduate                      1
8         27107        Graduate                      1
11        23853        Graduate                      1
12        25619        Graduate                      1
15         6588        Graduate                      1
20        31972         Masters                      2
21        19061         Masters                      2
23         7041        Graduate                      1
29        10408        Graduate                      1
30        14928        Graduate                      1
31        22293        Graduate                      1
33        26966        Graduate                      1
34        26494        Graduate                      1
35         4866        Graduate                      1
37        10164             Phd                      3
40        

In [49]:
# Explore data
job_df.describe()

,enrollee_id,city_development_index,training_hours,target,education_level_label
count,8955.000000,8955.000000,8955.000000,8955.000000,8955.000000
mean,16869.638749,0.844570,65.074930,0.165606,1.330207
std,9963.804718,0.116178,60.235087,0.371747,0.527190
min,2.000000,0.448000,1.000000,0.000000,1.000000
25%,8150.000000,0.794000,23.000000,0.000000,1.000000
50%,16924.000000,0.910000,47.000000,0.000000,1.000000
75%,25902.000000,0.920000,88.000000,0.000000,2.000000
max,33380.000000,0.949000,336.000000,1.000000,3.000000


## Codificando atributos categóricos nominales

#### Paso 2: se codifican las categorías numéricas para categorías nominales. 

Luego de convertir las categorías a representaciones numéricas hay que codificarlas para evitar que los modelos interpreten esas representaciones como valores contínuos. Por ejemplo, el caso de los géneros de los vídeo juegos, si se incluye directamente el atributo GenreLabel como una característica en un modelo de aprendizaje automático, este lo podría considerar como una característica numérica continua lo que podría no generar los resultados esperados. Por lo tanto, se requiere una capa adicional de codificación donde se crean características ficticias para cada categoría del atributo. 

### One-Hot Encoding

In [50]:
#Datos originales
print(job_df[['enrollee_id', 'education_level', 'education_level_label', 'gender']].head(20))

    enrollee_id education_level  education_level_label  gender
1         29725        Graduate                      1    Male
4           666         Masters                      2    Male
7           402        Graduate                      1    Male
8         27107        Graduate                      1    Male
11        23853        Graduate                      1    Male
12        25619        Graduate                      1    Male
15         6588        Graduate                      1    Male
20        31972         Masters                      2    Male
21        19061         Masters                      2    Male
23         7041        Graduate                      1    Male
29        10408        Graduate                      1    Male
30        14928        Graduate                      1    Male
31        22293        Graduate                      1    Male
33        26966        Graduate                      1  Female
34        26494        Graduate                      1 

In [51]:
# Check for NaN 
print(job_df.isnull().values.any())

False


In [52]:
# Transform and map the gender from categorical values to codes
# labelEncoder: encode target labels with value between 0 and classes-1.

#Transform the genre 
gender_le = LabelEncoder()
gender_labels = gender_le.fit_transform(job_df['gender'])

job_df['gender_Label'] = gender_labels

job_df_sub = job_df[['enrollee_id', 'education_level', 'education_level_label', 'gender', 'gender_Label']]

print(job_df[['enrollee_id', 'education_level', 
                 'education_level_label', 'gender', 'gender_Label']].head(10))


print("Verificación de nulos en cada columna")
print(job_df_sub.isnull().sum())

    enrollee_id education_level  education_level_label gender  gender_Label
1         29725        Graduate                      1   Male             1
4           666         Masters                      2   Male             1
7           402        Graduate                      1   Male             1
8         27107        Graduate                      1   Male             1
11        23853        Graduate                      1   Male             1
12        25619        Graduate                      1   Male             1
15         6588        Graduate                      1   Male             1
20        31972         Masters                      2   Male             1
21        19061         Masters                      2   Male             1
23         7041        Graduate                      1   Male             1
Verificación de nulos en cada columna
enrollee_id              0
education_level          0
education_level_label    0
gender                   0
gender_Label      

In [53]:

# One-Hot encoding is applied to gender_label 
genderLabel_ohe = OneHotEncoder()
genderLabel_feature_arr = genderLabel_ohe.fit_transform(
                              job_df_sub[['gender_Label']]).toarray()

# Class label names
genderLabel_feature_labels = list(gender_le.classes_)
genderLabel_features = pd.DataFrame(genderLabel_feature_arr, 
                            columns=genderLabel_feature_labels)



In [54]:
print(genderLabel_feature_labels)
print(gender_le.classes_)
print(genderLabel_feature_arr)

['Female', 'Male', 'Other']
['Female' 'Male' 'Other']
[[0. 1. 0.]
 [0. 1. 0.]
 [0. 1. 0.]
 ...
 [1. 0. 0.]
 [1. 0. 0.]
 [0. 1. 0.]]


In [55]:
# Organize data 
job_df_ohe = pd.concat([job_df_sub.reset_index(drop=True), 
                        genderLabel_features.reset_index(drop=True)], axis=1, ignore_index=True)

job_df_ohe.columns = ['enrollee_id', 'education_level', 'education_level_label', 
                      'gender', 'gender_Label', 'Female', 
                      'Male', 'Other']
print(job_df_ohe.head(20))

    enrollee_id education_level  education_level_label  gender  gender_Label  \
0         29725        Graduate                      1    Male             1   
1           666         Masters                      2    Male             1   
2           402        Graduate                      1    Male             1   
3         27107        Graduate                      1    Male             1   
4         23853        Graduate                      1    Male             1   
5         25619        Graduate                      1    Male             1   
6          6588        Graduate                      1    Male             1   
7         31972         Masters                      2    Male             1   
8         19061         Masters                      2    Male             1   
9          7041        Graduate                      1    Male             1   
10        10408        Graduate                      1    Male             1   
11        14928        Graduate         

### Referencias y más ejemplos

[1] Sarkar, D. (2018). Categorical Data.  Recuperado de https://towardsdatascience.com/understanding-feature-engineering-part-2-categorical-data-f54324193e63
    
    
[2] Here’s All you Need to Know About Encoding Categorical Data (with Python code). Recuperado de  https://www.analyticsvidhya.com/blog/2020/08/types-of-categorical-data-encoding/


[3] Formplus Blog. What is Ordinal Data? Examples, Variables & Analysis. Recuperado de https://www.formpl.us/blog/ordinal-data

[4] Scikit-learn (2016). Category Encoders.Recuperado de http://contrib.scikit-learn.org/category_encoders/index.html

[5] Dhasade, G (2020). Ways To Handle Categorical Data With Implementation. Recuperado de https://towardsdatascience.com/ways-to-handle-categorical-data-before-train-ml-models-with-implementation-ffc213dc84ec